---
**Project**: Cable Burial Operability  
**Notebook**: 01 – Generate Grid  
**Author**: Alejandra L. Cameselle  
**Date**: June 2025  

### Description
This notebook generates a fishnet grid over the Area of Interest (AOI) and assigns unique IDs to each cell. This grid will be used in subsequent analyses of bathymetry, slope and burial risk assessment.

### Inputs
- `aoi.gpkg`: AOI polygon
- `cell_size`: Desired spacing between grid cells in meters

### Outputs
- `01_grid.gpkg`: Regular grid clipped to AOI with cell ID

### Dependencies
- geopandas, shapely, matplotlib
---


In [1]:
# Import required libraries
import os
import numpy as np
import geopandas as gpd
from shapely.geometry import box

In [2]:
# Load the Area of Interest (AOI) from the GeoPackage as GeoDataFrame
aoi = gpd.read_file("../inputs/aoi.gpkg")

In [3]:
# Reproject to EPSG:25829 (ETRS89 / UTM zone 29N) if needed
if aoi.crs.to_epsg() != 25829:
    aoi = aoi.to_crs(epsg=25829)

In [4]:
# Define grid parameters
cell_size = 100  # in meters
minx, miny, maxx, maxy = aoi.total_bounds

In [5]:
# Create a list os square grid cells covering the bounding box
grid_cells = []
for x in np.arange(minx, maxx, cell_size):
    for y in np.arange(miny, maxy, cell_size):
        grid_cells.append(box(x, y, x + cell_size, y + cell_size))

In [6]:
# Convert list of boxes to a GeoDataFrame
grid = gpd.GeoDataFrame(geometry=grid_cells, crs=aoi.crs)

In [7]:
# Create mask column with centroids within AOI
aoi_union = aoi.geometry.union_all()
grid["inside_aoi"] = grid.geometry.centroid.within(aoi_union)

In [8]:
# Save result as GeoPackage
os.makedirs("../processed_data", exist_ok=True)
fname = f"../processed_data/01_grid_{cell_size}m.gpkg"
layer = f"01_grid_{cell_size}m"
grid.to_file(fname, layer=layer, driver="GPKG")

print(f"Grid saved with {len(grid)} cells — {grid['inside_aoi'].sum()} inside AOI")

Grid saved with 565468 cells — 540242 inside AOI
